**Day 0**

>Me: give me a super simple math problem to solve, we're talking like grade school arithmetic but formulate it like an illiterate peasant from the 15th century

>chatGPT: Ahoy! Me needeth thee to answereth this poser: if me have 3 apples and me give 2 to me mate, how many apples be left for me to enjoy with me ale?

<img src=https://cdn.historycollection.com/wp-content/uploads/2018/09/Peasants-pic-11-Wikimedia-Commons.jpg>

In [466]:
print(3-2)

"""proof"""
1 == 3-2

1


True

>Me: Ahoy! I reckon you will have one apple and your mate will have two! How generous!

>chatGPT: Ye be a clever one, matey! 'Tis correct! Ye have solved the poser with ease. Enjoy ye one apple and me hope ye find it to be a delicious treat!

OK that was stupid, but it's a proof of concept. I will be asking chatGPT (OR someone else/myself) for a math problem to solve with Python each day until whenever I'm fed up with this idea.

*Disclaimer: As I've previously used chatGPT and received some rather... creative calculations, I think it's a good idea to double check that the answers is indeed correct through other means.*

---

**Day 1**

> Me: Give me a very easy linear system of equations to solve.

>chatGPT: Sure, here's a simple one:

$x + y + z = 6$

$2x + 2y + 2z = 12$

$3x + 3y + 3z = 18$

<img src='https://wojakparadise.net/wojak/9450/img'>

Interesting, this look like some kind of arithmic sequence.
Just by looking at it I can tell there's an easy solution:
letting all coefficients (x,y,x) equal to 2.

Let's try it.

In [467]:
import numpy as np
coeff = np.array([[1, 1, 1],
                  [2, 2, 2],
                  [3, 3, 3]])
const = np.array( [6, 12, 18])

# Since we assume all coefficients have the same value, we can create a rather unconventional loop
for i in range(3):
    if coeff[i].sum() * 2 != const[i]:
        print(f"Equation {i+1} is not satisfied.")
    else:
        print(f"Equation {i+1} is satisfied.")
x,y,z = 2,2,2

# "Proof"
print(sum(coeff[0])*x == const[0], sum(coeff[1])*y == const[1], sum(coeff[2])*z == const[2])


Equation 1 is satisfied.
Equation 2 is satisfied.
Equation 3 is satisfied.
True True True


In [468]:
# Well, I'm glad that worked out, but what if I intuitively didn't 'see' the 2? How could I have satisfied the equations?
# Also, are there other possible solutions? I guess we could try to create some rrf.

print((coeff[0] * - 2) + coeff[1])
print((const[0] * - 2) + const[1])
print("hm, ok, this doesn't work, what about")
print((coeff[0] * - 1) + coeff[1])
print((const[0] * - 1) + const[1])
print("ok, now we just got the first equation")

# yeah imma get back to this, maybe

[0 0 0]
0
hm, ok, this doesn't work, what about
[1 1 1]
6
ok, now we just got the first equation


---

**Day 2**

>Me: Give me a challenge in probability by using Hearthstone as an example scenario. Perhaps some card drawing mechanics or lethal probability by drawing a certain card etc.

>chatGPT: Sure, here's a challenge for you!
 Let's say you're playing Hearthstone and you have a certain deck of 30 cards. In this deck, there are 4 legendary cards, 10 rare cards, 12 common cards and 4 basic cards.
 Challenge:
 What is the probability of drawing a legendary card in the first 3 draws of your deck, assuming the cards are not replaced once they have been drawn?

 <img src='https://liquipedia.net/commons/images/6/6b/Garrosh_Hellscream.png'>

In [469]:
# There are multiple ways (obviously) to aproach this challenge.

draw1, draw2, draw3 = 4/30, 4/29, 4/28
print(f'Probability of drawing a legendary card on the first three draws: {(draw1+draw2+draw3 * 100):.3f}%')

Probability of drawing a legendary card on the first three draws: 14.557%


---
**Day 3**

>Me: can u create another hearthstone challenge, any math is ok

<blockquote>chatGPT: Sure, here's a more specific Hearthstone challenge for you: <br><br>
Suppose you have a deck of 30 cards, containing 10 spells, 18 minions, and 2 weapons. The spell cards have a mana cost between 1 and 4 mana, the minions have a mana cost between 1 and 8 mana, and the weapons have a mana cost of 2 and 4 mana.

Challenge:

What is the probability of having a hand of 5 cards with a total mana cost of 10 or less, assuming that you draw 5 cards from your deck at the start of the game?

Hint: You may want to consider the number of cards with each mana cost, the number of different ways you can choose 5 cards with a total mana cost of 10 or less, and the different combinations of spells, minions, and weapons that can add up to 10 or less mana cost.</blockquote>

In [470]:
import numpy as np
np.random.seed(42) # to keep consistency when testing functions

# I decide  to pre-determine the mana cost for all cards
# we /could/ use some kind of wave function collapse for
# card mana values but that might be a bit of an overkill
# another aproach would be to not pre-determ values and sum ALL probabilities

# Anyway: randomize 30 cards within specified mana range for each type of card
random_cards = np.array([[np.random.randint(low=1, high=5, size=10)],   # Spells
                         [np.random.randint(low=1, high=9, size=18)],   # Minions
                         [np.random.randint(low=2, high=5, size =2)]],  # Weapons
                         dtype=object)

# list comprehenstion
deck = [random_cards[i][0].tolist() for i in range(len(random_cards))]

# check if everything is correct
print(f'With a deck of {len(deck[0])} spells, {len(deck[1])} minions and {len(deck[2])} weapons:')
print(f'Each spell card mana: {deck[0]}')
print(f'Each minion card mana: {deck[1]}')
print(f'Each weapon card mana: {deck[2]}')

# But since it doesn't matter what type of card we draw
only_mana = [foo for bar in deck for foo in bar] # readability /s

print(f'Deck still has 30 cards: {len(only_mana) == 30}')

With a deck of 10 spells, 18 minions and 2 weapons:
Each spell card mana: [3, 4, 1, 3, 3, 4, 1, 1, 3, 2]
Each minion card mana: [3, 7, 3, 3, 8, 5, 4, 8, 8, 3, 6, 5, 2, 8, 4, 6, 6, 2]
Each weapon card mana: [2, 2]
Deck still has 30 cards: True


In [471]:
# first let's just do a rough estimate
only_mana.sort() # sort list
print(f'{only_mana} sum = {sum(only_mana)} \n ')

mean = sum(only_mana)/len(only_mana)

print(f'{mean*5=}')

[1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 6, 6, 6, 7, 8, 8, 8, 8] sum = 120 
 
mean*5=20.0


In [472]:
# the average hand, using very simple est methods, would have a theoretical cost of 20 mana
# with this estimation, we can assume probability of a hand to have < 10 mana will be very low

# actually let's do some more statistics with pandas
import pandas as pd
df = pd.DataFrame({'Mana cost': only_mana})

df.describe().T

,count,mean,std,min,25%,50%,75%,max
Mana cost,30.0,4.0,2.212815,1.0,2.25,3.0,5.75,8.0


Let's get some notation in here because I need to become better at that.

**Binomial coefficient formula**
<h2> 

$n \choose k$ = $\frac{n!}{k!(n-k)!}$

</h2>

$n$ is used to calculate the number of **combinations** of size $k$ that can be selected from a set of $n$ items.

**Permutation formula**

<h2>

$_n P _r$ = $\frac{n!}{n-r!}$

</h2>

$n$ = total number of objects / set / population <br>
$r$ = number of objects selected / subset of n or sample set

Here $n$ is used to calculate the number of **arrangements** of $r$ items that can be selected from a set of $n$ items.


In [473]:
# the deck can be shuffled in 30! (factorial) combinations but we're interested in a hand of five cards

# Binomial coefficient formula: n choose k = n! / (k! * (n-k)!)
# we could import factorial function from math library or use scipy library but this should work as well

def fact(n):
    if n == 0: return 1
    else: return n * fact(n - 1)

def binom_coeff(n,k):
    return fact(n)/(fact(k) * fact(n-k))

n = 30 # cards in deck
k = 5  # cards in hand

print(f"There are {int(binom_coeff(n,k))} possivle combinations of {k} drawn cards in a deck of {n} cards. \n")

def permutation(n,r):
    return fact(n)/fact(n-r)

print(f"There are {int(permutation(n,k))} possible arrangements of {k} drawn cards in a deck of {n} cards. \n")

print('Given a partially randomized mana cost for each card, how many of these combinations add up to 10 or less mana?')



There are 142506 possivle combinations of 5 drawn cards in a deck of 30 cards. 

There are 17100720 possible arrangements of 5 drawn cards in a deck of 30 cards. 

Given a partially randomized mana cost for each card, how many of these combinations add up to 10 or less mana?


In [474]:
np.bincount(only_mana) # damn i need to learn more numpy

array([0, 3, 5, 8, 4, 2, 3, 1, 4])

**Cartesian product maybe can help**

In [475]:
# The brute force loop method - because Python is more fun than theoretical statistics (sry bout it x)
hand = only_mana

# might wanna use enumerate here...
count = 0

# Loop while the sum of mana cost from drawn cards >10
while sum(hand) > 10:
    # Update the list by randomly selecting a value to decrease by 1
    hand = np.random.choice(only_mana, size=5)
    print(f"Iteration {count+1}: {hand}")
    count += 1

print(f"Final hand: {hand}")
print(f"Number of iterations: {count +1}")


Iteration 1: [3 7 5 8 3]
Iteration 2: [6 4 8 8 3]
Iteration 3: [8 8 3 3 8]
Iteration 4: [8 3 8 4 3]
Iteration 5: [6 4 6 1 2]
Iteration 6: [4 2 5 3 2]
Iteration 7: [4 2 6 8 3]
Iteration 8: [4 7 3 7 5]
Iteration 9: [1 4 8 3 8]
Iteration 10: [2 3 8 2 3]
Iteration 11: [1 3 4 2 4]
Iteration 12: [2 2 1 8 2]
Iteration 13: [5 3 2 5 8]
Iteration 14: [4 7 3 1 3]
Iteration 15: [8 2 3 3 3]
Iteration 16: [2 3 6 8 6]
Iteration 17: [8 2 5 3 3]
Iteration 18: [4 3 5 6 7]
Iteration 19: [6 8 8 8 3]
Iteration 20: [3 8 3 3 1]
Iteration 21: [6 2 3 6 1]
Iteration 22: [3 2 6 3 4]
Iteration 23: [4 2 1 1 1]
Final hand: [4 2 1 1 1]
Number of iterations: 24


In [476]:
# ok now we just run this loop like a thousand times and check the values (this is so stupid...)
# let's quickly rewrite the the loop and make it callable

def nested_af():
    hand = only_mana
    # Initialize iteration count to 0
    count = 0

    # Loop while the sum of mana cost from drawn cards >10
    while sum(hand) > 10:
        # Update the list by randomly selecting a value to decrease by 1
        hand = np.random.choice(only_mana, size=5)
        count += 1

    # returns array with the iteration count and the final, <10 mana cost hand
    return count +1, hand


a, b = nested_af()
a, b # ok it works, let's run it

(7, array([1, 1, 2, 3, 3]))

In [482]:
all_iterations = []

for i in range(1000):
    a, b = nested_af()
    all_iterations.append(1/a)

# but since we wanna sum up the percentages
probs_dec = [i/100 for i in all_iterations]
prob_sum = sum(probs_dec)

prob_sum

0.50086100469601

In [483]:
# Ok, I got it to converge to like ~0.5% Perhaps should check it with a more valid method??

ANYWAY. My final answer is. 0.5%.